# GNSS simulation

## Constellation definition
We take here GALILEO constellation, with TLE dated 12/04/2022 at 13h00

In [17]:
from datetime import datetime, timezone
from datetime import tzinfo
import numpy as np
from blocksim.satellite.Satellite import createSatellites

t0 = datetime(
    year=2022, month=4, day=12, hour=13, minute=0, second=0, tzinfo=timezone.utc
)
sats = createSatellites(tle_file="galileo.tle", tsync=t0)
nsat = len(sats)

## GNSS tracker and receiver definition

In [18]:
from blocksim.gnss.GNSSTracker import GNSSTracker
from blocksim.gnss.GNSSReceiver import GNSSReceiver
from blocksim.utils import rad

tkr = GNSSTracker(name="tkr", nsat=nsat)
uere = 0.0
ueve = 0.0
tkr.elev_mask = rad(5.0)
tkr.dp = 300.0
tkr.dv = 50.0
cov = np.zeros((2 * nsat, 2 * nsat))
for k in range(nsat):
    cov[2 * k, 2 * k] = uere**2
    cov[2 * k + 1, 2 * k + 1] = ueve**2
tkr.setCovariance(cov, oname="measurement")

rec = GNSSReceiver(
    name="rec",
    nsat=nsat,
    lon=rad(43.60510103575826),
    lat=rad(1.4439216490854043),
    alt=0,
    tsync=t0,
)
rec.algo = "ranging"
rec.optim = "trust-constr"

## Simulation setup

In [19]:
from blocksim.Simulation import Simulation
from blocksim.control.Route import Group

nom_coord = ["px", "py", "pz", "vx", "vy", "vz"]

grp_snames = []
grp_inp = dict()
for k, sat in enumerate(sats):
    grp_inp["itrf%i" % k] = (6,)
    grp_snames.extend(["%s%i" % (n, k) for n in nom_coord])

# The Group so defined let us gather all the outputs of the satellites into one "wire" that feeds the tracker
grp = Group(
    "grp",
    inputs=grp_inp,
    snames=grp_snames,
)

sim = Simulation(*sats)
sim.addComputer(tkr)
sim.addComputer(rec)
sim.addComputer(grp)

# Then we do the connections
for k, sat in enumerate(sats):
    sim.connect("%s.itrf" % sat.getName(), "grp.itrf%i" % k)

sim.connect("rec.realpos", "tkr.ueposition")
sim.connect("grp.grouped", "tkr.state")
sim.connect("tkr.measurement", "rec.measurements")
sim.connect("tkr.ephemeris", "rec.ephemeris")

## Simulation and results

In [20]:
from blocksim.utils import itrf_to_geodetic, deg

tps = np.linspace(0, 3, 3)
sim.simulate(tps, progress_bar=False)

log = sim.getLogger()

# Computed ITRF position
x = log.getValue("rec_estpos_px")[-1]
y = log.getValue("rec_estpos_py")[-1]
z = log.getValue("rec_estpos_pz")[-1]
lon, lat, alt = itrf_to_geodetic((x, y, z))
print(deg(lon), deg(lat), alt)

# Estimate range bias
dp_est = log.getValue("rec_estclkerror_dp")[-1]
print(dp_est)

# Number of visible satellites
vissat = log.getValue("tkr_vissat_n")[-1]
print(vissat)

43.60510103575997 1.4439216555549719 4.168599843978882e-06
299.999997292144
12
